In [1]:
import re

In [2]:
def clean(q):
    if '/'  in q:
        indx1=q.find('/')
        q=q[:indx1]
    q=q.rstrip()
    q=q[::-1]
    q=q.rstrip()
    q=q[::-1]
    return q
def commandtype(q):
    type1=[  'push',  'pop',  'label',  'goto',  'if-goto', 'function',  'return',  'call',  'add','sub','neg','eq','gt','lt','and','or','not']
    type2=[  'C_PUSH','C_POP','C_LABEL','C_GOTO','C_IF',    'C_FUNCTION','C_RETURN','C_CALL','C_ARITHMETIC']
    counter=0
    index=None
    address=0
    q=re.split(r'\s+',q)
    for i in range(len(type1)):
        if type1[i]==q[0]:
            index=i
            break
    if  index <= 7:
        return type2[index]
    else :
        return type2[8]
    
    #for i in range(len(type1)):
    #    if q.find(type1[i])>= 0:
    #        counter=counter+1
    #        index=i
    #        address=q.find(type1[i])
    #if counter>1:
    #    print('more than one command type in one command')
    #elif  address != 0:
    #    print('command type is not in the first,syntax error')
    #elif  counter == 0:
    #    print('no command type')
    #elif   index <= 7: 
    #    return type2[index]
    #else :
    #    return type2[8]
def get_arg1(q,c):
    #q is the command need to parse
    #c is the commandtype corresponds to q
    if c == "C_ARITHMETIC":
        return q
    elif c== "C_RETURN":
        return "error,C_RETURN should not call arg"
    else:
        q=re.split(r'\s+',q)
        return q[1]
def get_arg2(q,c):
    #q is the command need to parse
    #c is the commandtype corresponds to q
    type3=['C_PUSH','C_POP','C_FUNCTION','C_CALL']
    if c in type3:
        q=re.split(r'\s+',q)
        return q[2]   
        return q
    else :
        return 'other type should not call arg2'
def constructor1(file):
    f=open(f,'r')
    text=f.readlines()
    for j in text:
        j.split('\n')[0]
        j=clean(j)

In [3]:
a=[]
a==[]

True

In [4]:
class codewriter():

    def __init__(self,filename):
        #re.split(r'\s+',command)
        self.filename=filename.split('.')[0]
        #print(filename)
        self.current_translated_file_name=filename.split('.')[0]
        self.label_num = 0
        self.current_function_name=[]
        self.if_in_function=[]
        self.return_label_num=0
        self.if_label_num=0
        self.text=[]
        
     #   f=open(self.filename + ".asm" ,"w")
    
    #def openfile(self):
     #   f=open(self.filename + ".asm" ,"w")
     #   f.write("aaaaaaaaaaaaaaaaaaa")
    
    #def closefile(self):
    #    f.close()
    def writebootstrap(self):
        self.writecode(['@256',
                     'D=A',
                     '@SP',
                     'M=D'])
        self.writecall("call Sys.init 0")
    
    
    
    
    
    def writeintofile(self):
        f=open(self.filename + ".asm" ,"w")
        for i in self.text:
            f.write(i+'\n')
        f.close()
        
    def get_new_if_label(self):
        self.if_label_num += 1
        return 'if_LABEL' + str(self.if_label_num)
    
    def get_real_label_name(self,label):
        
        if (self.if_in_function==[]):
            real_label=label
        else:
            real_label=self.current_function_name[-1]+ label
        return real_label
    
            
        
    def writecode(self,q):
        for i in q:
            #print(i)
            #f.write(i+'\n')
            self.text.append(i)
    def pointup(self):
        self.writecode(['@SP',
                    'M=M-1',
                     'A=M'])
    def write_push_from_d_register(self):
        self.writecode([
            '@SP',
            'A=M',
            'M=D',
            '@SP',
            'M=M+1',
            'A=M'    #add in 12.25 2021
        ])  
        
        
    def writebinarycommand(self,command):
            self.pointup()
            self.writecode(['D=M'])
            self.pointup()
            if command == "add":
                self.writecode(['D=M+D'])
            elif command == "sub":
                self.writecode(['D=M-D'])
                #self.writecode(['D=D-M'])
            elif command == 'and':
                self.writecode(['D=D&M'])
            elif command == 'or':
                self.writecode(['D=D|M'])            
            self.write_push_from_d_register()
    def writeunarycommand(self,command):
        self.pointup()
        self.writecode(['D=M'])
        if command == "neg":
            self.writecode(['D=-M'])
        if command == "not":
            self.writecode(['D=!M'])
        self.write_push_from_d_register()
    def writecompcommand(self,command):
        self.pointup()
        self.writecode(['D=M'])
        self.pointup()
        l1 = self.get_new_if_label()
        l2 = self.get_new_if_label()
        if command == "eq":
            comp_type = "JEQ"
        elif command == "gt":
            comp_type = "JGT"
        elif command == "lt":
            comp_type = "JLT"
        self.writecode([
            'D=M-D',
            "@%s" % l1,
            'D;%s' % comp_type,
            'D=0',
            "@%s" % l2,
            '0;JMP',
            "(%s)" % l1,
            'D=-1',
            "(%s)" % l2,
        ])
        self.write_push_from_d_register()        
        
        
        
        
    def writearithmetic(self,command):
        if command in ['add','sub','and','or']:
            self.writebinarycommand(command)
        elif  command in ['neg','not']:
            self.writeunarycommand(command)
        elif command in ['eq','gt','lt']:
            self.writecompcommand(command)    
    
        
            
    def writelabel(self,command):
        command=re.split(r'\s+',command)
        label=self.get_real_label_name(command[1])
        self.writecode(["(" +label+ ")"])
            
            
            
    def writegoto(self,command):
        command=re.split(r'\s+',command)
        label=command[1]
        label=self.get_real_label_name(label)
        self.writecode(['@%s' %label,
                        '0,JMP' ])
        #some thing need to be added
    
    def writeif(self,command):
        command=re.split(r'\s+',command)
        label=command[1]
        label=self.get_real_label_name(label)
        self.pointup()
        self.writecode(["D=M",
                  "@%s"%label,
                   "D,JNE"])
        
        
    def get_new_return_label(self):
        self.return_label_num += 1
        return '_RETURN_LABEL_' + str(self.return_label_num)           
        
    
        
    def writecall(self,command):
        
        command=re.split(r'\s+',command)
        functionname=command[1]
        argnumber=command[2]
        
        
        returnaddress=self.get_new_return_label()

        
        co2="push local 0"
        co3="push argument 0"
        co4="push this 0"
        co5="push that 0"
        
        
        
        self.writecode(['@%s'%returnaddress,
                        'D=A'])
        self.write_push_from_d_register()
        
        
        
        self.writecode(['@LCL',
                        'D=M' , 
                        '@SP' ,
                        'A=M' ,
                        'M=D' ,
                        '@SP' ,
                        'M=M+1'])

        
        self.writecode(['@ARG',
                        'D=M' , 
                        '@SP' ,
                        'A=M' ,
                        'M=D' ,
                        '@SP' ,
                        'M=M+1'])
        

        self.writecode(['@THIS',
                        'D=M' , 
                        '@SP' ,
                        'A=M' ,
                        'M=D' ,
                        '@SP' ,
                        'M=M+1'])

        self.writecode(['@THAT',
                        'D=M' , 
                        '@SP' ,
                        'A=M' ,
                        'M=D' ,
                        '@SP' ,
                        'M=M+1'])        
        #self.push(co2)  #  push local 0
        #self.push(co3)  #  push argument 0
        #self.push(co4)  # push this 0
        #self.push(co5)  #  push that 0
        
        
        self.writecode(['@SP',
                        'D=M'])
        for i in range(int(argnumber)+5):
            self.writecode(['D=D-1'])
            
            
        self.writecode(['@ARG',
                        'M=D'])    #ARG=SP-n-5
        
        self.writecode(['@SP',
                        'D=M',
                        '@LCL',
                        'M=D'])     #LCL=SP
        
        
        gotocommand='goto '+str(functionname)
        self.writegoto(gotocommand)
        returnaddress='label ' +returnaddress
        self.writelabel(returnaddress)
        
        
        
        
    def writefunctionhead(self,command):
        command=re.split(r'\s+',command)
        functionname=command[1]
        #print(functionname)
        argnumber=int(command[2])
        functionnamewrite='label '+functionname      #give a label type command

        self.writelabel(functionnamewrite)
        self.writecode(['@LCL',
                       'D=M',
                       '@SP',
                       'M=D'])
        self.writecode(['D=0'])
        for i in range(argnumber):
            self.write_push_from_d_register()
        
        
        if functionname != 'Sys.init':
            self.if_in_function.append(1)
            self.current_function_name.append(functionname)
        #print(self.current_function_name)
        #print(self.if_in_function)
        
    
     

    def writereturn(self,command):
        self.writecode(['@LCL',
                          'D=M',
                          '@R13',
                          'M=D',    # FRAME = LCL
                          '@5',
                          'D=D-A',
                          'A=D',
                          'D=M',
                          '@R14',
                          'M=D',
                        
                       #   '@R13',
                       #   'A=M-D',     # FRAME-5
                       #   'D=M',       # 存疑
                       #   #'D=A',        
                       #   '@R14',
                       #   'M=D',  # R14 = return-address ，# RET=*(FRAME-5)    
                          ])
        self.pointup()
        self.writecode([
            'D=M',
            '@ARG',
            'A=M',  # M = *ARG
            'M=D',  # *ARG = pop()

            '@ARG',
            'D=M+1',
            '@SP',
            'M=D',  # SP = ARG + 1
            
            
            '@R13',
            'D=M',  # A = FRAME-1, R13 = FRAME-1 #存疑
            'D=D-1',
            'A=D',
            'D=M',
            '@THAT',
            'M=D',  # THAT = *(FRAME-1)
            
            

            '@R13',
            'D=M',
            'D=D-1',
            'D=D-1',
            'A=D',
            'D=M',
            '@THIS',
            'M=D',  # THIS = *(FRAME-2)
             
            
            
            '@R13',
            'D=M',
            'D=D-1',
            'D=D-1',
            'D=D-1',
            'A=D',
            'D=M',
            '@ARG',
            'M=D',  # ARG = *(FRAME-3)
             
            
            '@R13',
            'D=M',
            'D=D-1',
            'D=D-1',
            'D=D-1',
            'D=D-1',
            'A=D',
            'D=M',
            '@LCL',
            'M=D',             # LCL = *(FRAME-4)
            
            
           # command='go to '
           # self.writegoto()
            '@R14',
            'A=M',
            '0;JMP'  # goto return-address
        ]
            
)
        self.if_in_function.pop()
        self.current_function_name.pop()
            
       
     
    
    def write_push_from_virtual_segment(self,command1,command2):
        if command1 == "local" :
            self.writecode(["@LCL"])
        elif command1=="argument":
            self.writecode(["@ARG"])
        elif command1=="this":
            self.writecode(["@THIS"])
        elif command1=="that":
            self.writecode(["@THAT"])
            
        self.writecode(["A=M"])
        
        for i in range( int(command2) ):
            self.writecode(['A=A+1'])
        self.writecode(["D=M"])   #存疑
        self.write_push_from_d_register()
    
    
    def write_push_from_static_segment(self,command1,command2):
        TEMP_BASE_ADDRESS=5
        POINTER_BASE_ADDRESS=3
        if command1=="pointer":
            address = POINTER_BASE_ADDRESS
            self.writecode(["@%s"% address]) 
        elif command1=="temp":
            address=TEMP_BASE_ADDRESS
            self.writecode(["@%s"% address])
        for i in range(int(command2) ):
            self.writecode(['A=A+1'])
        self.writecode(['D=M']) #存疑
        self.write_push_from_d_register()           
        
            
            
            
    def write_pop_to_virtual_segment(self,command1,command2):
        self.pointup()
        self.writecode(['D=M'])
        if command1=="local":
            virtual_address="LCL"
        elif command1 =="argument":
            virtual_address="ARG"
        elif command1=="this":
            virtual_address="THIS"
        elif command1=="that":
            virtual_address="THAT" 
        
        self.writecode(["@%s"%virtual_address,
                        "A=M"])
        for i in range( int(command2) ):
            self.writecode(['A=A+1'])
        self.writecode(["M=D"])
        
        
    def write_pop_to_static_segment(self,command1,command2):
        TEMP_BASE_ADDRESS=5
        POINTER_BASE_ADDRESS=3
        self.pointup()
        self.writecode(['D=M'])
        if command1 == "temp":
            real_address = TEMP_BASE_ADDRESS
        elif command1 == "pointer":
            real_address = POINTER_BASE_ADDRESS
        self.writecode([
            '@%s' % real_address,
        ])
        for i in range(int(command2)):
            self.writecode(['A=A+1'])
        self.writecode(['M=D'])        

        
    def pop(self,command):
        command=re.split(r'\s+',command)
        if command[1] in  ["local", "argument", "this", "that"]:
            self.write_pop_to_virtual_segment(command[1],command[2])
        elif command[1] in ["temp", "pointer"]:
            self.write_pop_to_static_segment(command[1],command[2])
        elif command[1] == "static":
               #print(self.if_in_function)
               # print(self.current_function_name)
                
                self.pointup()
                self.writecode(['D=M'])
                
                
                if (self.if_in_function[-1]):
                    name=self.current_function_name[-1].split('.')
                    self.writecode(["@%s.%s.%d" %(name[0],self.current_translated_file_name,int(command[2])),])
                else:    
                    self.writecode([
                    "@%s.%d" %(self.current_translated_file_name,int(command[2])),])
                #print(self.current_translated_file_name)
                
                
                self.writecode(['M=D'])
  
    
    def push(self,command):
        command=re.split(r'\s+',command)
        if command[1]=='constant':
            self.writecode(['@'+str(int(command[2])),'D=A'])
            self.write_push_from_d_register()
        elif command[1] in  ["local", "argument", "this", "that"]:
            self.write_push_from_virtual_segment(command[1],command[2])
        elif command[1] in ["temp", "pointer"]:
            self.write_push_from_static_segment(command[1],command[2])
        elif command[1] == "static":
                #self.writecode([
                #    "@%s.%d" % (self.current_translated_file_name,int(command[2])),
                #])
                
                if (self.if_in_function[-1]):
                    
                    name=self.current_function_name[-1].split('.')
                    self.writecode(["@%s.%s.%d" %(name[0],self.current_translated_file_name,int(command[2])),])
                else:    
                    self.writecode([
                    "@%s.%d" %(self.current_translated_file_name,int(command[2])),])            

            
                self.writecode(['D=M'])
                self.write_push_from_d_register()
                

In [8]:
fr1=open('Sys.vm')
q1=fr1.read()
q1=q1.split('\n')
m=codewriter("NestedCall.asm")
#name=fr.name.split('.')
#if name[0]!='Sys':
m.writebootstrap()
for i in q1:
    i=clean(i)
    
    if i=="":
        continue
    type1=commandtype(i)
    if type1=="C_PUSH":
        m.push(i)
    elif type1=="C_POP" :
        m.pop(i)
    elif type1=="C_ARITHMETIC":
        m.writearithmetic(i)
    elif type1=="C_LABEL":
        m.writelabel(i)
    elif type1=="C_GOTO":
        m.writegoto(i)
    elif type1=='C_IF':
        m.writeif(i)
    elif type1=="C_FUNCTION":
        m.writefunctionhead(i)
    elif type1=='C_RETURN':
        m.writereturn(i)
    elif type1=='C_CALL':
        m.writecall(i)        
        
m.writeintofile()

In [9]:
fr1.close()